In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trainMP=pd.read_csv(r'C:\Users\Nelson\Documents\Sync\Archivos de práctica Curso\trainMF.csv')# este archivo esta bien
trainMP=trainMP.drop('Unnamed: 0',axis=1)
trainMP

,Survived,Pclass,SibSp,Parch,Fare,Sex Encode,Embarked-2,Age-sep
0,0,3,1,0,7.250,1,2,1
1,1,1,1,0,712.833,0,0,3
2,1,3,0,0,7.925,0,2,2
3,1,1,1,0,53.100,0,2,3
4,0,3,0,0,8.050,1,2,3
...,...,...,...,...,...,...,...,...
886,0,2,0,0,13.000,1,2,2
887,1,1,0,0,30.000,0,2,1
888,0,3,1,2,23.450,0,2,2
889,1,1,0,0,30.000,1,0,2


In [3]:
trainME=trainMP.drop('Survived', axis=1)
y=trainMP['Survived'] 
X=trainME

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_test

,Pclass,SibSp,Parch,Fare,Sex Encode,Embarked-2,Age-sep
709,3,1,1,152.458,1,0,2
439,2,0,0,10.500,1,2,2
840,3,0,0,7.925,1,2,1
720,2,0,1,33.000,0,2,0
39,3,1,0,112.417,0,0,1
...,...,...,...,...,...,...,...
494,3,0,0,8.050,1,2,1
359,3,0,0,78.792,0,1,2
65,3,1,1,152.458,1,0,2
481,2,0,0,0.000,1,2,2


In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
param_grid = {'n_estimators'  : [50, 100, 500, 1000],
              'max_features'  : ['auto', 'sqrt', 'log2'],
              'max_depth'     : [None, 1, 3, 5, 10, 20],
              'subsample'     : [0.5, 1],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

grid = GridSearchCV(
        estimator  = GradientBoostingClassifier(random_state=42),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=42), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)


resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

,param_learning_rate,param_max_depth,param_max_features,param_n_estimators,param_subsample,mean_test_score,std_test_score,mean_train_score,std_train_score
347,0.1,3,sqrt,100,1,0.824278,0.018087,0.883086,0.005942
355,0.1,3,log2,100,1,0.824278,0.018087,0.883086,0.005942
337,0.1,3,auto,50,1,0.824268,0.021835,0.880447,0.002579
197,0.01,3,auto,500,1,0.822955,0.016903,0.883086,0.005942


In [6]:
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

{'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'n_estimators': 100, 'subsample': 1} : 0.8242779764518895 accuracy


In [7]:
modelo_final = grid.best_estimator_

In [8]:
modelo_final.score(X_train, y_train)

0.8652575957727873

In [9]:
modelo_final.score(X_test, y_test)#UN POCO MENOS QUE RNDOM FOREST

0.8432835820895522

In [11]:
predicciones = modelo_final.predict(X = X_test)

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [14]:
importancia_predictores = pd.DataFrame(
                            {'predictor': X_train.columns,
                             'importancia': modelo_final.feature_importances_}
                            )

importancia_predictores.sort_values('importancia', ascending=False)

,predictor,importancia
4,Sex Encode,0.512643
0,Pclass,0.164804
3,Fare,0.135701
6,Age-sep,0.072669
1,SibSp,0.051516
2,Parch,0.037705
5,Embarked-2,0.024963


In [ ]:
param_grid = {'n_estimators'  : [ 100],
              'max_features'  : ['sqrt'],
              'max_depth'     : [3],
              'subsample'     : [ 1],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

grid1 = GridSearchCV(
        estimator  = GradientBoostingClassifier(random_state=42),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=42), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid1.fit(X = X_train, y = y_train)

In [16]:
from sklearn.model_selection import RepeatedStratifiedKFold
def get_models():
    models = dict()
    for i in range(0.1, 2.1, 0.1):
        key = '%.3f' % i
        models[key] = AdaBoostClassifier(learning_rate=i)
    return models
 

def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

models = get_models()

results, names = list(), list()

for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)

    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.xticks(rotation=45)
pyplot.show()

TypeError: 'float' object cannot be interpreted as an integer